In [13]:
import numpy as np
from draw_roulette import draw_roulette

d = draw_roulette(list(np.random.randint(-2, 24, 1)), (24, 3), handles=5, phase=30, h_phase=15, random_seed=0)
quality = 280
d.setPixelScale(quality)  # Set number of pixels per geometry unit
d.rasterize()  # Display as PNG
d.savePng('cover.png')
d
